In [43]:
import os
from PIL import Image

input_directory = '/home/monika/HLA-TMA/HLA-TMA/1-2'
output_directory = '/home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped'

# Ensure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(input_directory):
    if "." not in filename:
        continue
    ending = filename.split(".")[1]
    if ending not in ["tif"]:
        continue

    try:
        image = Image.open(os.path.join(input_directory, filename))
    except IOError as e:
        print("Problem Opening", filename, ":", e)
        continue

    width, height = image.size

    # Determine the size of the square crop (use the smaller dimension)
    min_dimension = min(width, height)
    left = (width - min_dimension) // 2
    upper = (height - min_dimension) // 2
    right = left + min_dimension
    lower = upper + min_dimension

    image = image.crop((left, upper, right, lower))

    # Save the cropped image to the output directory
    output_filename = os.path.join(output_directory, filename)
    image.save(output_filename)
    
    print(f"Cropped and saved: {output_filename}")

print("Center cropping finished.")

Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/D10.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/E15.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/A8.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/F1.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/J10.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/A12.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/B14.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/B2.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/E10.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/B13.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/A6.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/A10.tif
Cropped and saved: /home/monika/HLA-TMA/HLA-TMA/cropped/1-2_cropped/E1.tif
Cropped and saved

In [35]:
from PIL import Image

input_directory = '/home/monika/HLA-TMA/HLA-TMA/1/A1.tif'

# Load the image
try:
    image = Image.open(input_directory)
    width, height = image.size
    print(f"Image size (width x height): {width} x {height}")
except IOError as e:
    print("Problem Opening", input_directory, ":", e)

Image size (width x height): 1920 x 1200


In [1]:
import os

# Directory to scan
directory = "/home/monika/HLA-TMA-cropped/cropped"

# Count images in each subdirectory and calculate 10%
image_counts = {}
for subdir, _, files in os.walk(directory):
    # Skip the root directory
    if subdir == directory:
        continue

    # Count only image files (assuming common image extensions)
    image_count = sum(1 for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.gif')))
    if image_count > 0:
        # Calculate 10%
        ten_percent = round(image_count * 0.1)
        image_counts[os.path.basename(subdir)] = {'total_images': image_count, '10_percent': ten_percent}

# Print the result
for class_name, counts in image_counts.items():
    print(f"Class '{class_name}': Total images = {counts['total_images']}, 10% = {counts['10_percent']}")

Class '4_cropped': Total images = 51, 10% = 5
Class '2_cropped': Total images = 84, 10% = 8
Class '0_cropped': Total images = 21, 10% = 2
Class '1_cropped': Total images = 24, 10% = 2
Class '3_cropped': Total images = 26, 10% = 3


In [5]:
from PIL import Image

input_directory = '/home/monika/HLA-TMA-10percent_to_train/cropped/0_cropped/D9.tif'

# Load the image
try:
    image = Image.open(input_directory)
    width, height = image.size
    print(f"Image size (width x height): {width} x {height}")
except IOError as e:
    print("Problem Opening", input_directory, ":", e)

Image size (width x height): 1200 x 1200


In [1]:
#APPLY CLAHE Contrast Limited Adaptive Histogram Equalization
!pip install opencv-python

import cv2
import os
import glob

def apply_clahe(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create a CLAHE object
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    
    # Process each sub-directory (class folder)
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)
        if os.path.isdir(class_path):
            output_class_dir = os.path.join(output_dir, class_folder)
            if not os.path.exists(output_class_dir):
                os.makedirs(output_class_dir)
            
            # Process each image in the class folder
            for img_path in glob.glob(os.path.join(class_path, '*')):
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # read image in grayscale
                if img is not None:
                    img_clahe = clahe.apply(img)  # apply CLAHE
                    output_path = os.path.join(output_class_dir, os.path.basename(img_path))
                    cv2.imwrite(output_path, img_clahe)  # save the processed image

input_directory = "/home/monika/HLA-TMA-cropped/cropped"
output_directory = "/home/monika/HLA_CLAHE"

apply_clahe(input_directory, output_directory)

In [4]:
#adaptive thresholding
import cv2
import os
import glob

def apply_adaptive_thresholding(image_path, output_path):
    # Read the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply adaptive thresholding
    thresholded_img = cv2.adaptiveThreshold(
        img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # Save the processed image
    cv2.imwrite(output_path, thresholded_img)

def process_images(input_dir, output_dir):
    # Check if output directory exists, if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over all class folders
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)
        output_class_path = os.path.join(output_dir, class_folder)

        # Check if class folder in output directory exists, if not, create it
        if not os.path.exists(output_class_path):
            os.makedirs(output_class_path)

        # Process all images in the class folder
        for image_file in glob.glob(os.path.join(class_path, '*.tif')):
            output_image_path = os.path.join(output_class_path, os.path.basename(image_file))
            apply_adaptive_thresholding(image_file, output_image_path)

# Define the input and output directories
input_directory = '/home/monika/HLA-TMA-cropped/cropped'
output_directory = '/home/monika/HLA_adaptTHRES'

# Process the images
process_images(input_directory, output_directory)

print("Adaptive thresholding applied to all images.")

Adaptive thresholding applied to all images.


In [5]:
import cv2
import os
import glob

def convert_to_hsl_and_save(input_image_path, output_image_path):
    # Read the image
    image = cv2.imread(input_image_path)

    # Convert to HSL color space
    hsl_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)

    # Save the converted image
    cv2.imwrite(output_image_path, hsl_image)

def process_images(input_dir, output_dir):
    # Check if the output directory exists, if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over all class folders
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)
        output_class_path = os.path.join(output_dir, class_folder)

        # Check if class folder in output directory exists, if not, create it
        if not os.path.exists(output_class_path):
            os.makedirs(output_class_path)

        # Process all images in the class folder
        for image_file in glob.glob(os.path.join(class_path, '*.*')):  # Adjust the pattern as needed
            output_image_path = os.path.join(output_class_path, os.path.basename(image_file))
            convert_to_hsl_and_save(image_file, output_image_path)

# Define the input and output directories
input_directory = '/home/monika/HLA-TMA-cropped/cropped'
output_directory = '/home/monika/HLA_hsl'

# Process the images
process_images(input_directory, output_directory)

print("HSL color space conversion applied to all images.")


HSL color space conversion applied to all images.


In [6]:
import cv2
import os
import glob

def convert_to_hsv_and_save(input_image_path, output_image_path):
    # Read the image
    image = cv2.imread(input_image_path)

    # Convert to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Save the converted image
    cv2.imwrite(output_image_path, hsv_image)

def process_images(input_dir, output_dir):
    # Check if the output directory exists, if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over all class folders
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)
        output_class_path = os.path.join(output_dir, class_folder)

        # Check if class folder in output directory exists, if not, create it
        if not os.path.exists(output_class_path):
            os.makedirs(output_class_path)

        # Process all images in the class folder
        for image_file in glob.glob(os.path.join(class_path, '*.*')):  # Adjust the pattern as needed
            output_image_path = os.path.join(output_class_path, os.path.basename(image_file))
            convert_to_hsv_and_save(image_file, output_image_path)

# Define the input and output directories
input_directory = '/home/monika/HLA-TMA-cropped/cropped'
output_directory = '/home/monika/HLA_hsv'

# Process the images
process_images(input_directory, output_directory)

print("HSV color space conversion applied to all images.")

HSV color space conversion applied to all images.


In [7]:
import cv2
import os
import glob

def convert_to_lab_and_save(input_image_path, output_image_path):
    # Read the image
    image = cv2.imread(input_image_path)

    # Convert to Lab color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)

    # Save the converted image
    cv2.imwrite(output_image_path, lab_image)

def process_images(input_dir, output_dir):
    # Check if the output directory exists, if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over all class folders
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)
        output_class_path = os.path.join(output_dir, class_folder)

        # Check if class folder in output directory exists, if not, create it
        if not os.path.exists(output_class_path):
            os.makedirs(output_class_path)

        # Process all images in the class folder
        for image_file in glob.glob(os.path.join(class_path, '*.*')):  # Adjust the pattern as needed
            output_image_path = os.path.join(output_class_path, os.path.basename(image_file))
            convert_to_lab_and_save(image_file, output_image_path)

# Define the input and output directories
input_directory = '/home/monika/HLA-TMA-cropped/cropped'
output_directory = '/home/monika/HLA_labcolor'

# Process the images
process_images(input_directory, output_directory)

print("Lab color space conversion applied to all images.")

Lab color space conversion applied to all images.
